In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('valorant.sqlite')

# Load data from the database into a pandas DataFrame
# Using the table names based on the dataset structure from Kaggle
# Reference: https://www.kaggle.com/datasets/visualize25/valorant-pro-matches-full-data
query_matches = "SELECT * FROM Matches"
query_game = "SELECT * FROM Games"
query_game_rounds = "SELECT * FROM Game_Rounds"
query_game_scoreboard = "SELECT * FROM Game_Scoreboard"

df_matches = pd.read_sql_query(query_matches, conn)
df_game = pd.read_sql_query(query_game, conn)
df_game_rounds = pd.read_sql_query(query_game_rounds, conn)
df_game_scoreboard = pd.read_sql_query(query_game_scoreboard, conn)

# Close the connection
conn.close()

In [2]:
df_matches.head()

,MatchID,Date,Patch,EventID,EventName,EventStage,Team1ID,Team2ID,Team1,Team2,Team1_MapScore,Team2_MapScore
0,62393,2022-01-08 15:30:00,Patch 3.12,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
1,62403,2022-01-08 15:30:00,Patch 3.12,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (C),7046,7047,Bjor's Kittens,Mugiwara,2,0
2,62391,2022-01-08 12:30:00,Patch 3.12,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Winner's (A),6461,6903,Akrew,Booster Seat Gaming,2,1
3,62396,2022-01-08 12:30:00,Patch 3.12,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Winner's (B),6164,7043,Radiance,sameROFLMAO,2,0
4,62401,2022-01-08 12:30:00,Patch 3.12,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Winner's (C),7045,7046,Salt and Vinegar,Bjor's Kittens,2,0


In [3]:
df_game.head()

,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_TotalRounds,Team2_TotalRounds,...,Team1_FullBuyWon,Team2_PistolWon,Team2_Eco,Team2_EcoWon,Team2_SemiEco,Team2_SemiEcoWon,Team2_SemiBuy,Team2_SemiBuyWon,Team2_FullBuy,Team2_FullBuyWon
0,60894,62393,Breeze,6903,6020,Booster Seat Gaming,Pho Real,Booster Seat Gaming,13,7,...,8.0,0.0,4.0,0.0,2.0,0.0,4.0,1.0,10.0,6.0
1,60895,62393,Bind,6903,6020,Booster Seat Gaming,Pho Real,Pho Real,2,13,...,1.0,2.0,2.0,2.0,0.0,0.0,4.0,3.0,9.0,8.0
2,60896,62393,Haven,6903,6020,Booster Seat Gaming,Pho Real,Booster Seat Gaming,13,8,...,9.0,1.0,2.0,1.0,2.0,0.0,6.0,2.0,11.0,5.0
3,60924,62403,Icebox,7046,7047,Bjor's Kittens,Mugiwara,Bjor's Kittens,13,6,...,8.0,0.0,4.0,0.0,1.0,0.0,2.0,1.0,12.0,5.0
4,60925,62403,Haven,7046,7047,Bjor's Kittens,Mugiwara,Bjor's Kittens,13,9,...,11.0,1.0,3.0,2.0,3.0,0.0,4.0,3.0,12.0,4.0


In [22]:
df_game_rounds.head()
print(df_game_rounds.iloc[288])


GameID          60217
Team1ID          6451
Team2ID          5482
RoundHistory     None
Name: 288, dtype: object


In [31]:
for i in range(1, 21):
    print(type(df_game_rounds["RoundHistory"][0]))

# Convert the string representation of the dictionary into an actual dictionary
import ast

all_round_histories = []

# Iterate through each row in df_game_rounds to extract and convert 'RoundHistory'
print(len(df_game_rounds))
for i in range(len(df_game_rounds)):
    # Extract the RoundHistory string from the DataFrame
    round_history_string = df_game_rounds.loc[i, 'RoundHistory']
       
    print(round_history_string)
    if round_history_string is not None:
        # Convert the string to a dictionary of dictionaries
        round_history_entries = ast.literal_eval(round_history_string)
        round_history_dict = {}




        #!!!!! TODO!!!!!! 
        #CONVERT TO DICTIONARY OF DICTIONARIES. 
        #outer dictionary is the game: keys = round number, value = round info
        #inner dictionay is the round info. Each Catergory corresponds to a key, value corresponds to the data 
        # this will make it easier to get put the info we need into our all_features array




        for entry in round_history_entries:                         
            sub_dict = {}
            for key, value in entry.items():
                sub_dict[key.strip()] = value.strip()
            round_number = entry['RoundNumber'].strip()
            round_history_dict[round_number] = sub_dict
        
        # Append the dictionary to the list
        all_round_histories.append(round_history_dict)
    else:
        print("Missing data from game index: ", i)

#all_round_histories is an array of maps. Each map corresponds to a game
#There is an entry in all_round_histories for every game that has info on the round history
#Each dictionary is formatted as such:
#Key: Round Number
#Value: The information on that given round(separated by commas)
#Example:
# 'RoundWinner': 'BOOS', 
# 'ScoreAfterRound': '1-0', 
# 'WinType': 'elim', 
# 'Team1Bank': 300.0, 
# 'Team2Bank': 100.0, 
# 'Team1BuyType': 'eco', 
# 'Team2BuyType': 'eco'

print(len(all_round_histories))



<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
15531
{1: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '1-0', 'WinType': 'elim', 'Team1Bank': 300.0, 'Team2Bank': 100.0, 'Team1BuyType': 'eco', 'Team2BuyType': 'eco'}, 2: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '2-0', 'WinType': 'elim', 'Team1Bank': 2700.0, 'Team2Bank': 8600.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'eco'}, 3: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '3-0', 'WinType': 'elim', 'Team1Bank': 12300.0, 'Team2Bank': 400.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'full-buy'}, 4: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '4-0', 'WinType': 'elim', 'Team1Bank': 23600.0, 'Team2Bank': 9800.0, 'Team1BuyType': 'full-buy', 'Team2BuyType': 'semi-eco'}, 5: {'RoundWinner': 'PHO ', 'ScoreAfterRound':

AttributeError: 'int' object has no attribute 'items'

In [27]:
#Write test code here:
print("Missing info from ", len(df_game_rounds) - len(all_round_histories), "out of", len(df_game_rounds), "games")
# Print all elements in the dictionary all_round_histories in order
if all_round_histories:
    for game_id, game_round_history in all_round_histories.items():
        print(f"Game ID: {game_id}")
        for round_number in sorted(game_round_history.keys()):
            print(f"Round {round_number}: {game_round_history[round_number]}")
else:
    print("No round histories available.")




Missing info from  677 out of 15531 games
Round 1: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '1-0', 'WinType': 'elim', 'Team1Bank': 300.0, 'Team2Bank': 100.0, 'Team1BuyType': 'eco', 'Team2BuyType': 'eco'}
Round 2: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '2-0', 'WinType': 'elim', 'Team1Bank': 2700.0, 'Team2Bank': 8600.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'eco'}
Round 3: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '3-0', 'WinType': 'elim', 'Team1Bank': 12300.0, 'Team2Bank': 400.0, 'Team1BuyType': 'semi-buy', 'Team2BuyType': 'full-buy'}
Round 4: {'RoundWinner': 'BOOS', 'ScoreAfterRound': '4-0', 'WinType': 'elim', 'Team1Bank': 23600.0, 'Team2Bank': 9800.0, 'Team1BuyType': 'full-buy', 'Team2BuyType': 'semi-eco'}
Round 5: {'RoundWinner': 'PHO ', 'ScoreAfterRound': '4-1', 'WinType': 'boom', 'Team1Bank': 24200.0, 'Team2Bank': 3300.0, 'Team1BuyType': 'full-buy', 'Team2BuyType': 'full-buy'}
Round 6: {'RoundWinner': 'PHO ', 'ScoreAfterRound': '4-2', 'WinType': 'boom', 'Team1Bank': 10

In [28]:
#FEATURES
# Previous round winner(1 or 2), 
# Num of Rounds already won by team1
# Num of Rounds already won by team2
#Team1Bank
#Team2Bank
#Team1BuyType(1,2,3)        [1=eco,2=semi-eco, 3=semi-buy,4=full buy]
#Team2BuyType(1,2,3)

#LABELS
#Round Winner(team1 or team2)


#!!!!!! ALL THIS CODE IS WRONG !!!!!!!
#dont touch unless u gonna fix all of it 
#THIS IS THE AI TEXT ENTRY

# For every map in all_round_histories, place the information from each entry in that map into a dataframe called all_features. Here is the info I want for a given entry:
# We will not be placing entries for round1 of any game, as there is no data on the previous round.
# 1. Previous round winner(1 or 2), For all entries other than round1, the previous round winner will be RoundWinner from the previous round. Currenty, the RoundWinner is a string. So to determine who is team1 and team2, look at ScoreAfterRound for round1. If the score is '1-0' we know the RoundWinner for round1 is team 1. If the score is '0-1', we know the RoundWinner for round1 is team 2.
# 2.The number of rounds already won by team1. Look at the ScoreAfterRound string of the previous round. The number on the left side of the dash will be the number of rounds already won by team1
# 3. The number of rounds already won by team2. Look at the ScoreAfterRound string of the previous round. The number on the right side of the dash will be the number of rounds already won by team2
# 4.Team 1 Bank
# 5. Team 2 Bank
# 6. Team 1 buy type (1, 2, 3, or 4)  [1=eco,2=semi-eco, 3=semi-buy,4=full buy]
# 7. Team 2 buy type (1,2,3, or4)     [1=eco,2=semi-eco, 3=semi-buy,4=full buy]

# No entries in the dataframe all_features, should be a string unless you want to add a header 


#!!TODO!!!!
# make it easy to convert team names into team1 or team2 using a dictionary
# this will make it so we can easily figure out whether team1 won or team2 won 
# based on the name of the winning team
# clean up the code so it does what we want




import pandas as pd

# Define a function to convert buy type from string to integer
def buy_type_to_int(buy_type):
    buy_mapping = {'eco': 1, 'semi-eco': 2, 'semi-buy': 3, 'full buy': 4}
    return buy_mapping.get(buy_type, 0)

# Initialize the dataframes
all_features = pd.DataFrame(columns=[
    'PreviousRoundWinner', 'Team1RoundsWon', 'Team2RoundsWon', 
    'Team1Bank', 'Team2Bank', 'Team1BuyType', 'Team2BuyType'
])
all_labels = pd.DataFrame(columns=['RoundWinner'])

# Process each game's round history
for curr_game in all_round_histories:
    # Determine initial team order from round 1 score
    initial_score = curr_game[1]['ScoreAfterRound'].split('-')
    if initial_score[0] == '1':
        team1_first = True
        
        nameToTeamNum = { #use the name of the team as the key, team num as the value
            "<TEAM1> ": 1,
            "<TEAM2>": 2
        }

    else:
        team1_first = False
        #create some kind of asociation between the team of RoundWinner(string) and 2



    # Iterate over each round, skipping the first round
    for round_number, round_info in curr_game.items():
        if round_number == '1':
            continue
        
        # Previous round number as string
        previous_round_number = str(int(round_number) - 1)
        previous_round_info = curr_game[previous_round_number]
        
        # Previous round winner
        if previous_round_info['RoundWinner'] == 'BOOS':
            previous_round_winner = 1 if team1_first else 2
        else:
            previous_round_winner = 2 if team1_first else 1
        
        # Rounds won by each team
        previous_score = previous_round_info['ScoreAfterRound'].split('-')
        team1_rounds_won = int(previous_score[0])
        team2_rounds_won = int(previous_score[1])
        
        # Current round info
        team1_bank = round_info['Team1Bank']
        team2_bank = round_info['Team2Bank']
        team1_buy_type = buy_type_to_int(round_info['Team1BuyType'])
        team2_buy_type = buy_type_to_int(round_info['Team2BuyType'])
        
        # Determine current round winner
        if round_info['RoundWinner'] == 'BOOS':
            current_round_winner = 1 if team1_first else 2
        else:
            current_round_winner = 2 if team1_first else 1
        
        # Append to dataframes
        all_features = all_features.append({
            'PreviousRoundWinner': previous_round_winner,
            'Team1RoundsWon': team1_rounds_won,
            'Team2RoundsWon': team2_rounds_won,
            'Team1Bank': team1_bank,
            'Team2Bank': team2_bank,
            'Team1BuyType': team1_buy_type,
            'Team2BuyType': team2_buy_type
        }, ignore_index=True)
        
        all_labels = all_labels.append({
            'CurrentRoundWinner': current_round_winner
        }, ignore_index=True)




all_features.head()
all_labels.head()

KeyError: '1'